In [1]:
import pandas as pd

In [2]:
# читаем изначальные данные
df_sales = pd.read_csv('sample.csv')

# переводим Date_buy из srt в Datetime формат
df_sales['Date_buy'] = pd.to_datetime(df_sales['Date_buy'])

In [3]:
%%time

#все вместе занимает 7-8 минут


#в изначальном датасете добавляем колонку с порядковым номером недели в году
df_sales['week_id'] = [df_sales.Date_buy[i].isocalendar()[1]-1 for i in range(df_sales.shape[0])]

#добавляем колонку с номером недели и номером дня в неделе в изначальный датасет
df_sales['Date_buy_week_day'] = [str(df_sales.Date_buy[i].isocalendar()[1]-1)+' '+
                                     str(df_sales.Date_buy[i].weekday()) for i in range(df_sales.shape[0])]


for item_id in df_sales.ID_SKU.unique():
    
    #складываем кол-во продаж по номеру недели и по номеру ID товара
    df_sales_weeks = df_sales[df_sales.ID_SKU == item_id].groupby('week_id').SKU_QTY.sum().reset_index()
    
    
    
    #складываем кол-во продаж по номеру неделии и дня недели, и по номеру ID товара
    df_sales_weeks_days = df_sales[df_sales.ID_SKU == item_id].groupby('Date_buy_week_day').SKU_QTY.sum().reset_index()
    
    
    #разбиваем датасет с номером недели и дня недели на [номер недели x номер дня в неделе]
    df_sales_weeks_days_wide = pd.DataFrame()
    
    for i in range(int(df_sales_weeks_days.shape[0]/7)):
        dict_new = dict()
        for j in range(7):
            dict_new['{}'.format(j)] = df_sales_weeks_days.SKU_QTY[i+j]
        
        df_sales_weeks_days_wide = df_sales_weeks_days_wide.append(dict_new, ignore_index=True)
    
    
    #объединяем два датасета: изначальный с кол-во продаж определенного товара за неделю 
    #и датасет [номер недели x номер дня в неделе] с кол-во продаж
    
    df_sales_weeks_days_wide_final = pd.DataFrame()
    df_sales_weeks_days_wide_final = pd.concat([df_sales_weeks, df_sales_weeks_days_wide], axis=1,sort=False)
    
    #добавляем колонку с номером ID товара
    df_sales_weeks_days_wide_final['ID_SKU'] = [item_id for j in range(df_sales_weeks_days_wide_final.shape[0])]
    
    #сохраняем получившийся датасет
    df_sales_weeks_days_wide_final.to_csv('df_sales_weeks_days_wide_final_id{}.csv'.format(item_id), index=False)

KeyboardInterrupt: 